# Deep Reinforcement Learning - Assignment 2
## Training and Comparing REINFORCE Agents on CartPole-v1

This notebook implements, trains, and evaluates Policy Gradient agents.

### Assignment Structure:
- **Section 1**: Vanilla REINFORCE Agent
- **Section 2**: REINFORCE with Baseline (Value Function)
- **Section 3**: Comparison and Analysis

---

In [ ]:
# Setup: Add project root to Python path
import os
import sys

# Get the notebook's directory
notebook_dir = os.path.abspath('')
project_root = notebook_dir  # Since notebook is in project root

# Add to path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any, List
from datetime import datetime
import random

# Gymnasium
import gymnasium as gym

# PyTorch
import torch

# Force complete module reload
import sys
modules_to_remove = [key for key in list(sys.modules.keys()) 
                    if 'agent' in key.lower() or 'src' in key or 'utils' in key or 'ffnn' in key]
for module in modules_to_remove:
    del sys.modules[module]

# Import custom agents
from src.agent import ReinforceAgent, ReinforceBaselineAgent

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# --- SEED SETTING ---
SEED = 42
def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seeds()
print(f"✓ Seeds set to {SEED}")
print(f"✓ PyTorch version: {torch.__version__}")

## Helper Functions

In [ ]:
def plot_training_metrics(results: Dict[str, Any], agent_name: str, save_path: str = None):
    """Plot rewards and losses."""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Rewards
    rewards = results['rewards']
    window = 100
    axes[0].plot(rewards, alpha=0.6, label='Episode Reward')
    if len(rewards) >= window:
        moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
        axes[0].plot(np.arange(window-1, len(rewards)), moving_avg, color='red', linewidth=2, label='100-Ep Avg')
    
    axes[0].axhline(y=475, color='green', linestyle='--', label='Target (475)')
    axes[0].set_title(f'{agent_name}: Rewards')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Reward')
    axes[0].legend()
    
    # Loss
    if results['loss']:
        axes[1].plot(results['loss'], color='orange', alpha=0.6)
        axes[1].set_title(f'{agent_name}: Loss')
        axes[1].set_xlabel('Episode')
        axes[1].set_ylabel('Loss')
    
    plt.tight_layout()
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
    plt.show()

def save_results(agent, results, name, base_dir="models"):
    """Save model weights and results."""
    os.makedirs(base_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save model state dict
    torch.save(agent.policy_net.state_dict(), f"{base_dir}/{name}_{timestamp}_policy.pth")
    if hasattr(agent, 'value_net'):
        torch.save(agent.value_net.state_dict(), f"{base_dir}/{name}_{timestamp}_value.pth")
        
    # Save metrics
    np.savez(f"{base_dir}/{name}_{timestamp}_results.npz", **results)
    print(f"✓ Saved {name} to {base_dir}/")

## Environment Setup

In [ ]:
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"State Dim: {state_dim}, Action Dim: {action_dim}")

---
# Section 1: Vanilla REINFORCE

In [ ]:
# Configuration
config_reinforce = {
    'gamma': 0.99,
    'learning_rate': 0.001,
    'hidden_dims': [128, 128]
}

# Initialize Agent
set_seeds(SEED) # Reset seeds for fair comparison
agent_reinforce = ReinforceAgent(state_dim, action_dim, config_reinforce)

print("Initialized Vanilla REINFORCE Agent")

In [ ]:
# Train
print("Starting Training...")
results_reinforce = agent_reinforce.train(
    env, 
    max_episodes=3000, 
    target_reward=475.0, 
    window=100
)

print(f"Training finished. Converged: {results_reinforce['converged']}")

In [ ]:
# Analyze
plot_training_metrics(results_reinforce, "Vanilla REINFORCE", "results/reinforce_training.png")
save_results(agent_reinforce, results_reinforce, "reinforce")

---
# Section 2: REINFORCE with Baseline

In [ ]:
# Configuration (Same hyperparameters for fair comparison)
config_baseline = {
    'gamma': 0.99,
    'learning_rate': 0.001,
    'hidden_dims': [128, 128]
}

# Initialize Agent
set_seeds(SEED) # Reset seeds
agent_baseline = ReinforceBaselineAgent(state_dim, action_dim, config_baseline)

print("Initialized REINFORCE with Baseline Agent")

In [ ]:
# Train
print("Starting Training...")
results_baseline = agent_baseline.train(
    env, 
    max_episodes=3000, 
    target_reward=475.0, 
    window=100
)

print(f"Training finished. Converged: {results_baseline['converged']}")

In [ ]:
# Analyze
plot_training_metrics(results_baseline, "REINFORCE w/ Baseline", "results/baseline_training.png")
save_results(agent_baseline, results_baseline, "baseline")

---
# Section 3: Comparison

In [ ]:
# Compare Rewards
plt.figure(figsize=(12, 6))

# Calculate moving averages
window = 100
r_reinforce = results_reinforce['rewards']
r_baseline = results_baseline['rewards']

ma_reinforce = np.convolve(r_reinforce, np.ones(window)/window, mode='valid')
ma_baseline = np.convolve(r_baseline, np.ones(window)/window, mode='valid')

plt.plot(ma_reinforce, label='Vanilla REINFORCE', alpha=0.8)
plt.plot(ma_baseline, label='REINFORCE w/ Baseline', alpha=0.8)

plt.axhline(y=475, color='green', linestyle='--', label='Target')
plt.title('Comparison: Moving Average Reward (Window=100)')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.legend()
plt.savefig("results/comparison.png")
plt.show()

print(f"Vanilla Converged at: {results_reinforce['episodes_trained']}")
print(f"Baseline Converged at: {results_baseline['episodes_trained']}")